# Predictor-Corrector WB2i as a Difference of Fluxes

In [ ]:
#For the LaTeX equations (such as eqnarray) in this document to work, include the following in file
#~/.jupyter/_config.yml
#
#parse:
#  myst_enable_extensions:  # default extensions to enable in the myst parser. See https://myst-parser.readthedocs.io/en/latest/using/syntax-optional.html
#     - amsmath
#
#(the default ~/.jupyter/_config.yml will have amsmath commented out)
%matplotlib inline
import numpy as np
from numpy import exp
import sympy as sy
from sympy import latex
from sympy import I   # symbol for sqrt of -1
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

## WB2i as fluxes

In [ ]:
# Derivation of WB2 using spatial gradients from a quadratic
# ax^2 + bx + c  so that ddx = 2ax + b, d2/dx2 = 2a
Psi = sy.Matrix(sy.symarray("Ψ", (3,)), real=True)    # Grid point values at i-2, i-1 and i
C = sy.symbols("C", real=True) # The Courant number

def findDdxD2dx2():
    PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (3,))) # Coefficients a,b,c of the polynomial
    polyM = Matrix([[4,-2,1], [1,-1,1], [0,0,1]])
    PolyCoeffs = polyM.solve(sy.Matrix(Psi))
    Ddx = PolyCoeffs[1]
    D2dx2 = 2*PolyCoeffs[0]
    return Ddx, D2dx2

WB_Ddx, WB_D2dx2 = findDdxD2dx2()

# From this we can work out WB2i
WB2i = sy.collect(sy.expand(- C*WB_Ddx - C**2/2*WB_D2dx2), Psi)
print('WB2i = ', WB2i)

# Find Ψ_l and Ψ_r so that WB2i = -Ψ_r + Ψ_l
WB2iR = WB2i # The residual of WB2i after removal of -Ψ_r + Ψ_l
[WBil, WBir] = sy.symbols("WBil, WBi4", real=True)
WBil = WBir = sy.S.Zero
for j in range(2):
    WBil = WBil + Psi[j]*WB2iR.coeff(Psi[j])
    WBir = WBir + Psi[j+1]*WB2iR.coeff(Psi[j])
    WB2iR = sy.collect(sy.expand(WB2i + WBir - WBil), Psi)

print('Residual =', WB2iR, '\nThe right flux is')
WBir

## WB2i as a predictor-corrector scheme

In [ ]:
WBrCorr = sy.collect(sy.expand(WBir - C*Psi[2]), Psi)
WBlCorr = sy.collect(sy.expand(WBil - C*Psi[1]), Psi)
WBlCorr

## Amplification Factor

In [ ]:
[Kdx, Alp] = sy.symbols("Kdx, Alp", real=True)
Acorr = WBrCorr - WBlCorr
Acorr = Acorr.subs({Psi[0] : sy.E**(-2*sy.I*Kdx), Psi[1] : sy.E**(-sy.I*Kdx), Psi[2] : 1})
Aup = 1/(1 + C*(1 - sy.E**(-sy.I*Kdx)))
# A series of amplification factors for increasing iteration count, m
A = sy.symarray("A", 8)
A[0] = sy.S.One
for m in range(1, len(A)):
    A[m] = Aup*(1 - A[m-1]*Acorr)

# A for fully implicit (to checkU)
Ai = 1/(1 + C*(1 - sy.E**(-sy.I*Kdx)) + Acorr)

In [ ]:
# Plots of |A| for 6 each iterations
kdxs = np.linspace(1e-6, 2*np.pi, 37)
cs = np.arange(0, 5.1, 0.1)
magA = np.zeros([len(kdxs), len(cs)])
for m in range(1,7):
    i = (m-1)%3
    if i == 0:
        fig,axs = plt.subplots(1,3, figsize=(12,4), layout='constrained')
        fig.suptitle("Predictor-Corrector Warming and Beam Amplification Factor Magnitudes")    
    A_WB = sy.lambdify([C, Kdx], A[m], 'numpy')
    for ic in range(len(cs)):
        co = cs[ic]
        for ik in range(len(kdxs)):
            kdx = kdxs[ik]
            magA[ik,ic] = abs(A_WB(co, kdx))
    axplot = axs[i].contourf(cs, kdxs,magA, np.arange(0, 2.1, 0.1))
    axs[i].axvline(x=1, color="black", linestyle=":")
    axs[i].axvline(x=2, color="black", linestyle=":")
    fig.colorbar(axplot,ax=axs[i], orientation='horizontal')
    axs[i].contour(cs, kdxs, magA, [0, 1], colors=['k', 'k'])
    axs[i].set(xlabel=r'$c$', ylabel=r'$k\Delta x$', title = str(m)+' iterations')

    if m%3 == 0:
        plt.show()

# Fully implicit scheme
Ai_WB = sy.lambdify([C,Kdx], Ai, 'numpy')
for ic in range(len(cs)):
    co = cs[ic]
    for ik in range(len(kdxs)):
        kdx = kdxs[ik]
        magA[ik,ic] = abs(Ai_WB(co, kdx))
plt.contourf(cs, kdxs,magA, np.arange(0, 2.1, 0.1))
plt.axvline(x=1, color="black", linestyle=":")
plt.axvline(x=2, color="black", linestyle=":")
plt.colorbar(orientation='horizontal')
plt.contour(cs, kdxs, magA, [0, 1], colors=['k', 'k'])
plt.xlabel(r'$c$')
plt.ylabel(r'$k\Delta x$')
plt.title("Fully implicit")
plt.show()

In [ ]:
# The PC version is always most unstable for kdx=pi
# so work out the limiting for the HO correction for kdx = pi

Chi = sy.symbols("Chi", real=True) # HO limiter
Acorr = Chi*(WBrCorr - WBlCorr)
Acorr = Acorr.subs({Psi[0] : sy.E**(-2*sy.I*Kdx), Psi[1] : sy.E**(-sy.I*Kdx), Psi[2] : 1})
Acorr = Acorr.subs({Kdx: sy.pi})
Aup = 1/(1 + C*(1 - sy.E**(-sy.I*sy.pi)))
# A series of amplification factors for increasing iteration count, m
A = sy.symarray("A", 8)
A[0] = sy.S.One
for m in range(1, len(A)):
    A[m] = Aup*(1 - A[m-1]*Acorr)

# Plot A for each iteration for kdx = pi
chis = np.arange(0, 1.01, 0.1)
magA = np.zeros([len(chis), len(cs)])
for m in range(1,7):
    i = (m-1)%3
    if i == 0:
        fig,axs = plt.subplots(1,3, figsize=(12,4), layout='constrained')
        fig.suptitle("Predictor-Corrector Warming and Beam Amplification Factor Magnitudes")    
    A_WB = sy.lambdify([C, Chi], A[m], 'numpy')
    for ic in range(len(cs)):
        co = cs[ic]
        for ich in range(len(chis)):
            chi = chis[ich]
            magA[ich,ic] = abs(A_WB(co, chi))
    axplot = axs[i].contourf(cs, chis,magA, np.arange(0, 2.1, 0.1))
    axs[i].axvline(x=1, color="black", linestyle=":")
    axs[i].axvline(x=2, color="black", linestyle=":")
    fig.colorbar(axplot,ax=axs[i], orientation='horizontal')
    axs[i].contour(cs, chis, magA, [0, 1], colors=['k', 'k'])
    axs[i].set(xlabel=r'$c$', ylabel=r'$\chi$', title = str(m)+' iterations')
    
    if m%3 == 0:
        plt.show()